In [166]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

DIR = Path().resolve()
PROJECT_ROOT = DIR.parent

sys.path.append(str(PROJECT_ROOT))

from src.eda.data_reader import DataReader


In [167]:
reader = DataReader()

df = reader.read()

In [168]:
class DataProcessor():
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        
    def clean_price(self):
        self.df['price'] = pd.to_numeric(self.df['price'], errors='coerce')
        self.df = self.df.dropna(subset=['price'])
    
        return self
    
    def clean_ppm(self):
        self.df.loc[:,'price_per_meter'] = self.df['price_per_meter'].fillna(self.df['price'] / self.df['area'])
        self.df['price_per_meter'] = pd.to_numeric(self.df['price_per_meter'], errors='coerce')

        return self

    def clean_rooms(self):
        self.df.loc[:, 'rooms'] = pd.to_numeric(self.df['rooms'], errors='coerce')
        self.df = self.df.dropna(subset=['rooms'])

        return self

    def clean_market_type(self):
        self.df = self.df.dropna(subset=['market_type'])

        return self

    def clean_furnished(self):
        self.df = self.df.dropna(subset=['furnished'])

        return self

    def clean_district(self):
        self.df = self.df.dropna(subset=['district'])

        return self

    def clean_building_type(self):
        self.df['building_type'] = self.df['building_type'].fillna('unknown')
        
        return self

    def clean_year_built(self):
        self.df.loc[self.df['year_built'] < 1300, 'year_built'] = np.nan
        self.df.loc[:,'year_built'] = self.df['year_built'].fillna(self.df['year_built'].median())
        
        return self

    def clean_rent(self):
        self.df.loc[:,'rent'] = self.df['rent'].fillna(self.df['rent'].median())

        return self

    def clean_finish_status(self):
        self.df = self.df.dropna(subset=['finish_status'])

        return self

    def clean_ownership(self):
        self.df = self.df.dropna(subset=['ownership'])

        return self

    def clean_heating(self):
        self.df.loc[:,'heating'] = self.df['heating'].fillna('unknown')

        return self

    def clean_elevator(self):
        self.df = self.df.dropna(subset=['elevator'])

        return self

    def clean_outliers(self, column, lower=0.05, upper=0.95):
        q_low = self.df[column].quantile(lower)
        q_high = self.df[column].quantile(upper)
        
        self.df.loc[:,column] = self.df[column].clip(lower=q_low, upper=q_high)

        return self

    def drop_duplicates(self):
        self.df = self.df.drop_duplicates(subset=['price', 'url'])
        self.df = self.df.reset_index(drop=True)

        return self

    def drop_columns(self, columns: [str]):
        self.df = self.df.drop(columns, axis=1)

        return self

data_processor = (DataProcessor(df)
      .clean_price()
      .clean_ppm()
      .clean_rooms()
      .clean_market_type()
      .clean_furnished()
      .clean_district()
      .clean_building_type()
      .clean_year_built()
      .clean_rent()
      .clean_finish_status()
      .clean_ownership()
      .clean_heating()
      .clean_elevator()
      .clean_outliers('price')
      .clean_outliers('price_per_meter')
      .clean_outliers('rent')
      .drop_duplicates()
      .drop_columns(['source', 'date', 'url', 'title', 'ad_id', 'external_id'])
     )

df = data_processor.df
df

,price,price_per_meter,area,rooms,floor,market_type,furnished,description,district,building_type,year_built,rent,finish_status,ownership,heating,elevator
0,944000.0000,19667.0000,48.0000,2.0000,parter/4,wtórny,Nie,<p> Z przyjemnością przedstawiam Państwu ...,śródmieście,tenement,1957.0000,700.0000,ready_to_use,full_ownership,miejskie,Nie
1,799000.0000,12292.0000,65.0000,4.0000,3/10,wtórny,Nie,"Mam przyjemność zaprezentować Państwu, mieszka...",praga-północ,block,1978.0000,520.0000,to_renovation,limited_ownership,miejskie,Nie
2,1993000.0000,29011.2000,46.8800,2.0000,5/8,pierwotny,Nie,<ul><li>2-pokojowe mieszkanie<strong> numer A....,wola,unknown,2026.0000,700.0000,to_completion,full_ownership,unknown,Nie
3,1333000.0000,12250.9000,134.0000,5.0000,1/2,wtórny,Nie,"<p>Na sprzedaż piękne, <strong>5- pokojowe, be...",ursynów,block,2016.0000,700.0000,ready_to_use,full_ownership,gazowe,Nie
4,1301000.0000,23560.0000,55.2200,3.0000,5/6,pierwotny,Nie,<ul><li>3-pokojowe mieszkanie<strong> numer AA...,wola,unknown,2026.0000,700.0000,to_completion,full_ownership,unknown,Nie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11040,890000.0000,24205.0000,36.7700,2.0000,3/7,wtórny,Nie,<p>Sprzedam bezpośrednio.</p>\r\n<p>2-pokojowe...,wola,block,2016.0000,341.0000,ready_to_use,full_ownership,miejskie,Nie
11041,525000.0000,17500.0000,30.0000,1.0000,5/7,wtórny,Nie,<p>Ładne mieszkanie do zamieszkania od zaraz ....,ursus,block,2008.0000,700.0000,ready_to_use,full_ownership,gazowe,Nie
11042,525000.0000,14045.0000,37.3800,2.0000,3/3,wtórny,Nie,<p>Sprzedaż bezpośrednia.</p>\r\n<p>2-pokojowe...,praga-północ,tenement,1950.0000,700.0000,ready_to_use,full_ownership,miejskie,Nie
11043,748467.0000,17698.0000,42.2900,2.0000,5/5,wtórny,Nie,<p>Oferta bezpośrednia nie współpracuje z biur...,bemowo,block,2025.0000,700.0000,to_renovation,full_ownership,unknown,Nie


In [169]:
import pandas as pd
import numpy as np
import sys
from sklearn.preprocessing import StandardScaler
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

class FeatureEngineer():
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.scalers = {}
        self.encodings = {}

    @staticmethod
    def load(path: str):
        print(f"Loading PreProcessor from: {path}")
        
        return joblib.load(path)

    def save(self, path: str):
        joblib.dump(self, path)
        print(f"PreProcessor saved to: {path}")

    def process_floor(self):
        #split floor column by sep "/"
        self.df[['floor', 'building_max_floor']]  = self.df['floor'].str.split('/', expand=True)
        
        floor_map = {
        'suterena': -1,
        'parter': 0,
        }
    
        self.df['floor'] = self.df['floor'].replace(floor_map)
    
        self.df['building_max_floor'] = pd.to_numeric(self.df['building_max_floor'], errors='coerce')
        self.df.loc[df['floor'] == 'poddasze', 'floor'] = self.df['building_max_floor'] + 1
            
        self.df['is_above_10_floor'] = self.df['floor'].astype(str).str.contains('>').astype(int)
    
        self.df.loc[:,'floor'] = self.df['floor'].astype(str).str.replace('>', '', regex=False)
    
        self.df['floor'] = pd.to_numeric(self.df['floor'], errors='coerce')

        self.df.loc[self.df['building_max_floor'] > 60, 'building_max_floor'] = np.nan
        
        self.df = self.df.dropna(subset=['floor', 'building_max_floor'])
    
        return self

    def has_elevator_in_desc(self):
        has_elevator = self.df['description'].str.contains(r'\bwinda\w*\b', case=False, na=False)
        self.df.loc[has_elevator, 'elevator'] = 1

        self.df['elevator'] = self.df['elevator'].replace({'Tak': 1, 'Nie': 0})
        self.df['elevator'] = self.df['elevator'].fillna(0).astype(int)
    
        return self

    def has_balcony_in_desc(self):
        has_balcony = self.df['description'].str.contains(r'\b(balkon\w*|taras\w*)\b', case=False, na=False)
        self.df.loc[has_balcony, 'balcony'] = 1

        self.df['balcony'] = self.df['balcony'].fillna(0)
    
        return self


    def has_garage_in_desc(self):
        has_garage = self.df['description'].str.contains(r'\bgaraż\w*\b', case=False, na=False)
        self.df.loc[has_garage, 'garage'] = 1
        self.df['garage'] = self.df['garage'].fillna(0)
    
        return self

    def has_furniture_in_desc(self):
        has_furniture = self.df['description'].str.contains( r'\bmeble\w*\b', case=False, na=False)
        self.df.loc[has_furniture, 'furnished'] = 1

        self.df['furnished'] = self.df['furnished'].replace({'Tak': 1, 'Nie': 0})
        self.df['furnished'] = self.df['furnished'].fillna(0).astype(int)
    
        return self
        
    def create_luxury_col(self):
        self.df["is_luxury"] = (self.df["price_per_meter"] > self.df["price_per_meter"].quantile(0.90)).astype(int)

        return self

    def create_room_per_area_col(self):
        self.df["rooms_per_area"] = (self.df["rooms"] / self.df["area"]).astype(float)

        return self

    def reduce_to_binary(self, column: str, positive_value: str, new_name: str = None):
        if new_name is None:
            new_name = column
        self.df[new_name] = (self.df[column] == positive_value).astype(int)
        
        return self

    def frequency_encoding(self, column: str):
        freq_map = self.df[column].value_counts(normalize=True)

        self.encodings[column] = freq_map.to_dict()
        
        self.df[f'{column}_freq'] = self.df[column].map(freq_map)
        
        return self

    def one_hot_encode(self, column: str, drop_original: bool = True):
        dummies = pd.get_dummies(self.df[column], prefix=column, drop_first=True, dtype=int)
        self.df = pd.concat([self.df, dummies], axis=1)
        if drop_original:
            self.df.drop(columns=[column], inplace=True)
        return self

    def reduce_building_type(self):
        top_types = {'apartment', 'block', 'tenement'}
        self.df['building_type'] = self.df['building_type'].apply(
            lambda x: x if x in top_types else 'other'
        )
        return self

    def drop_columns(self, columns: [str]):
        self.df = self.df.drop(columns, axis=1)
        self.df = self.df.reset_index(drop=True)
        
        return self

    def scale_columns(self, columns: list, method='standard'):
        for col in columns:
            if col not in self.df.columns:
                print(f"[SKIP] Column '{col}' not found.")
                continue

            scaler = StandardScaler()
            self.df[col] = scaler.fit_transform(self.df[[col]])
            self.scalers[col] = scaler

        print(f"Scaled columns: {columns}")
        return self

feature_engineer = (FeatureEngineer(df)
    .process_floor()
    .has_elevator_in_desc()
    .has_balcony_in_desc()
    .has_garage_in_desc()
    .has_furniture_in_desc()
    .create_luxury_col()
    .create_room_per_area_col()
    .reduce_to_binary('heating' , 'miejskie', 'district_heating')
    .reduce_to_binary('ownership' , 'full_ownership', 'full_ownership')
    .reduce_to_binary('market_type' , 'pierwotny', 'primary_market')
    .reduce_building_type()
    .frequency_encoding('district')
    .one_hot_encode('finish_status')
    .one_hot_encode('building_type')
    .drop_columns(['description', 'ownership', 'heating', 'market_type', 'district'])
    .scale_columns(['price', 'price_per_meter', 'area', 'year_built', 'rent', 'rooms', 'floor', 'building_max_floor'])
     )

feature_engineer.save("features_engineer_v0.pkl")
df = feature_engineer.df

df

C:\Users\Jakub\AppData\Local\Temp\ipykernel_19380\1986575474.py:53: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['elevator'] = self.df['elevator'].replace({'Tak': 1, 'Nie': 0})
C:\Users\Jakub\AppData\Local\Temp\ipykernel_19380\1986575474.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  has_balcony = self.df['description'].str.contains(r'\b(balkon\w*|taras\w*)\b', case=False, na=False)


Scaled columns: ['price', 'price_per_meter', 'area', 'year_built', 'rent', 'rooms', 'floor', 'building_max_floor']
PreProcessor saved to: features_engineer_v0.pkl


C:\Users\Jakub\AppData\Local\Temp\ipykernel_19380\1986575474.py:78: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df['furnished'] = self.df['furnished'].replace({'Tak': 1, 'Nie': 0})


,price,price_per_meter,area,rooms,floor,furnished,year_built,rent,elevator,building_max_floor,...,rooms_per_area,district_heating,full_ownership,primary_market,district_freq,finish_status_to_completion,finish_status_to_renovation,building_type_block,building_type_other,building_type_tenement
0,-0.2112,0.2875,-0.3769,-0.5892,-1.1451,1,-0.4054,-0.0595,0,-0.4993,...,0.0417,1,1,0,0.0876,0,0,0,0,1
1,-0.4864,-1.3168,0.1995,1.4686,0.0161,0,-0.1990,-0.6125,0,0.8673,...,0.0615,1,0,0,0.0322,0,1,1,0,0
2,1.7805,2.3202,-0.4149,-0.5892,0.7903,0,0.2729,-0.0595,0,0.4118,...,0.0427,0,1,1,0.1101,1,0,0,1,0
3,0.5274,-1.3258,2.5389,2.4974,-0.7580,0,0.1746,-0.0595,0,-0.9548,...,0.0373,0,1,0,0.0495,0,0,1,0,0
4,0.4666,1.1344,-0.1321,0.4397,0.7903,0,0.2729,-0.0595,0,-0.0438,...,0.0543,0,1,1,0.1101,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10450,-0.3137,1.2747,-0.7577,-0.5892,0.0161,0,0.1746,-1.1625,0,0.1840,...,0.0544,1,1,0,0.1101,0,0,1,0,0
10451,-1.0067,-0.1839,-0.9872,-1.6180,0.7903,0,0.0959,-0.0595,0,0.1840,...,0.0333,0,1,0,0.0332,0,0,1,0,0
10452,-1.0067,-0.9355,-0.7370,-0.5892,0.0161,0,-0.4743,-0.0595,0,-0.7271,...,0.0535,1,1,0,0.0322,0,0,0,0,1
10453,-0.5824,-0.1408,-0.5705,-0.5892,0.7903,0,0.2630,-0.0595,0,-0.2715,...,0.0473,0,1,0,0.0475,0,1,1,0,0
